# Chart generations Notebook

## Import Libraries

In [1]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as offline
import os
import numpy as np
from plotly.offline import plot

In [2]:
offline.init_notebook_mode(connected=True)

## Bubble Chart for Emissions Intensity


In [3]:
corp_name = "Accenture"

# Step 1: Set up the data sources

In [4]:
XLSX_PATH = os.path.join ('../django_project/static/django_project', 'data', 'sp100_data.xlsx')
COLS_TO_USE = {
    'companies':['company_id', 'company_name'],
    'GHG19':['company_id', 'gross_total_scope1', 'gross_loc_scope2', 'gross_mkt_scope2'],
    'grouping':['company_id', 'Sector1'],
    'financials':['company_id', 'Revenue']
}


# Step 2: Merge the tables

In [5]:
all_df = []
for sheetname, cols in COLS_TO_USE.items():
    all_df.append(pd.read_excel(
        XLSX_PATH, 
        sheet_name = sheetname,
        engine = 'openpyxl', 
        usecols = cols
        ))
#all_data.to_pickle (os.path.join ('..','data_frame.pickle'))

In [6]:
merged_df = all_df[0]
for i in range(len(COLS_TO_USE) - 1):
    merged_df = pd.merge(
        left = merged_df,
        right = all_df[i+1],
        how="left",
        on="company_id"
    )

In [7]:
merged_df.head()

,company_id,company_name,gross_total_scope1,gross_loc_scope2,gross_mkt_scope2,Sector1,Revenue
0,1,3M,4050000,1780000,1320000,Industrials,3.213600e+10
1,2,Abbott Laboratories,533000,518000,439000,Healthcare,3.190400e+10
2,3,AbbVie Inc.,314421,308204,249777,Healthcare,3.326600e+10
3,4,Accenture,18923,281489,214680,Technology,4.320000e+10
4,5,Adobe Inc.,11816,56113,43526,Technology,1.117100e+10


# Step 3: Filter the sector

In [8]:
sector = merged_df[merged_df['company_name']==corp_name]['Sector1'].iloc[0]

In [9]:
merged_df_sector = merged_df.loc[merged_df['Sector1']==sector]

# Calculate GHG Intensity

In [16]:
merged_df_sector = merged_df_sector.assign(scope1_plus_scope2= (merged_df_sector['gross_total_scope1'] + merged_df_sector['gross_mkt_scope2']))

In [17]:
merged_df_sector.assign(intensity=(merged_df_sector['scope1_plus_scope2'] / merged_df_sector['Revenue']))

,company_id,company_name,gross_total_scope1,gross_loc_scope2,gross_mkt_scope2,Sector1,Revenue,scope1_plus_scope2,intensity
3,4,Accenture,18923,281489,214680,Technology,4.320000e+10,233603,5.40748e-06
4,5,Adobe Inc.,11816,56113,43526,Technology,1.117100e+10,55342,4.95408e-06
13,14,Apple Inc.,50549,862127,0,Technology,2.601740e+11,50549,1.94289e-07
26,27,Cisco Systems,41181,651331,187428,Technology,5.190000e+10,228609,4.4048e-06
52,53,IBM Corp.,117723,987066,827369,Technology,7.714000e+10,945092,1.22516e-05
53,54,Intel Corp.,1489000,3345000,1299000,Technology,7.190000e+10,2788000,3.87761e-05
65,66,Microsoft Corp.,113412,3556553,275375,Technology,1.258000e+11,388787,3.09052e-06
71,72,Nvidia,2817,74692,65936,Technology,1.172000e+10,68753,5.8663e-06
73,74,Oracle,16520,560683,349022,Technology,3.950000e+10,365542,9.25423e-06
79,80,QUALCOMM Inc.,78290,135967,114060,Technology,2.427000e+10,192350,7.92542e-06


# Fetch specific values for the selected company

In [11]:
corp_x = merged_df_sector[merged_df_sector['company_name']==corp_name]['gross_total_scope1'].iloc[0]
corp_y = merged_df_sector[merged_df_sector['company_name']==corp_name]['Revenue'].iloc[0]

# Fetch data series for the bubble chart data

In [16]:
x0 = merged_df_sector['gross_total_scope1']
y0 = merged_df_sector['Revenue']


In [17]:
"""
upper_left_ann = dict (xref="x domain",
                       yref="paper",
                       x=0.10,
                       y=1.10,
                       text="<b>Corporate Behemoths</b> <br> High Revenue, High Emissions",
                       showarrow = False,
                       bgcolor = 'blue',
                       font = {'color':'white'},
                       opacity = 0.5
                      )
upper_right_ann = dict (xref="x domain",
                       yref="paper",
                       x=0.90,
                       y=1.10,
                       text="<b>Sustainability Leaders</b> <br> High Revenue, Low Emissions",
                       showarrow = False,
                       bgcolor = 'blue',
                       font = {'color':'white'},
                       opacity = 0.5
                       )
lower_right_ann = dict (xref="x domain",
                       yref="paper",
                       x=0.90,
                       y=-0.15,
                       text="<b>Small Players</b> <br> Low Revenue, Low Emissions",
                       showarrow = False,
                       bgcolor = 'blue',
                       font = {'color':'white'},
                       opacity = 0.5
                       )
lower_left_ann = dict (xref="x domain",
                       yref="paper",
                       x=0.10,
                       y=-0.15,
                       text="<b>Worst Offenders</b> <br> Low Revenue, High Emissions",
                       showarrow = False,
                      bgcolor = 'blue',
                       font = {'color':'white'},
                       opacity = 0.5
                      )
"""

'\nupper_left_ann = dict (xref="x domain",\n                       yref="paper",\n                       x=0.10,\n                       y=1.10,\n                       text="<b>Corporate Behemoths</b> <br> High Revenue, High Emissions",\n                       showarrow = False,\n                       bgcolor = \'blue\',\n                       font = {\'color\':\'white\'},\n                       opacity = 0.5\n                      )\nupper_right_ann = dict (xref="x domain",\n                       yref="paper",\n                       x=0.90,\n                       y=1.10,\n                       text="<b>Sustainability Leaders</b> <br> High Revenue, Low Emissions",\n                       showarrow = False,\n                       bgcolor = \'blue\',\n                       font = {\'color\':\'white\'},\n                       opacity = 0.5\n                       )\nlower_right_ann = dict (xref="x domain",\n                       yref="paper",\n                       x=0.90,\n 

In [18]:
layout = go.Layout (
    title = 'Sector:' + sector + 'GHG Emissions Intensity',
    title_x = 0.5,
    titlefont = dict(family = 'Arial', size = 25),
    plot_bgcolor = 'antiquewhite',
    xaxis =  dict(autorange = "reversed", type = 'log'),
    yaxis = dict(type = 'log'),
    #annotations = [upper_left_ann, upper_right_ann, lower_left_ann, lower_right_ann]
)

In [19]:
x_median_x = [x0.min(), x0.max()]
y_median_x = [y0.median(), y0.median()]
x_median_y = [x0.median(), x0.median()]
y_median_y = [y0.min(), y0.max()]

In [24]:
index_corp = merged_df_sector.loc[merged_df_sector['company_name'] == corp_name].index
# Delete these row indexes from dataFrame
merged_df_sector.drop(index_corp , inplace=True)

C:\Users\vince\anaconda3\envs\cnext\lib\site-packages\pandas\core\frame.py:4170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
merged_df_sector

,company_id,company_name,gross_total_scope1,gross_loc_scope2,gross_mkt_scope2,Sector1,Revenue
4,5,Adobe Inc.,11816,56113,43526,Technology,1.117100e+10
13,14,Apple Inc.,50549,862127,0,Technology,2.601740e+11
26,27,Cisco Systems,41181,651331,187428,Technology,5.190000e+10
52,53,IBM Corp.,117723,987066,827369,Technology,7.714000e+10
53,54,Intel Corp.,1489000,3345000,1299000,Technology,7.190000e+10
65,66,Microsoft Corp.,113412,3556553,275375,Technology,1.258000e+11
71,72,Nvidia,2817,74692,65936,Technology,1.172000e+10
73,74,Oracle,16520,560683,349022,Technology,3.950000e+10
79,80,QUALCOMM Inc.,78290,135967,114060,Technology,2.427000e+10
81,82,Salesforce,5747,290938,134760,Technology,1.328000e+10


In [25]:
index_corp

Int64Index([3], dtype='int64')

In [30]:
trace1 = go.Scatter(x=x_median_y, 
                    y=y_median_y, 
                    showlegend = False, 
                    name='Median y',
                    mode = "lines",
                    line = dict(color='gray', 
                                width=2, 
                                dash='dash'))

trace2 = go.Scatter(x=x_median_x, 
                         y=y_median_x, 
                         showlegend = False, 
                         name='Median x', 
                         mode = "lines",
                         line = {'color':'gray', 
                                 'width':2, 
                                 'dash':'dash'}
                        )

trace3 = go.Scatter(
    x=merged_df_sector['gross_total_scope1'], 
    y=merged_df_sector['Revenue'],
    showlegend = False,
    name="benchmark", 
    text=merged_df_sector['company_name'],
    mode = 'markers+text',
    line = {'color':'black'}, 
    marker = {
        'color': 'blue',
        'opacity': 0.5,
        'size':20
    },
    textposition='top center'
    )
trace4 = go.Scatter(
    x=[corp_x], 
    y=[corp_y],
    showlegend = False,
    name=corp_name, 
    text=corp_name,
    mode = 'markers+text',
    line = {'color':'black'}, 
    marker = {
        'color': 'red',
        'opacity': 1,
        'size':20
    },
    textposition='top center'
    )

fig = go.Figure(data = [trace1, trace2, trace3, trace4], 
                layout = layout)
                
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)
config = {'displaylogo': False}


In [31]:
#num_traces_no_markers = 2
#indexes = list(range(num_traces_no_markers,len(sector_names) + num_traces_no_markers - 1))


In [32]:
# Add dropdown
"""
fig.update_layout(
    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                dict(
                    args=["mode", "markers", indexes],
                    label="Hide names",
                    method="restyle"
                ),
                dict(
                    args=["mode", "markers+text", indexes],
                    label="Show names",
                    method="restyle"
                )
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=1.0,
            xanchor="left",
            y=1.25,
            yanchor="top"
        ),
    ]
)
"""
offline.iplot(fig, config = config)


In [22]:
name_fig = "bubble_intensity_{}".format(corp_name)
fig.write_image("../django_project/static/django_project/images/charts/{}.svg".format(name_fig), scale=3, height = 300)
plot(fig, config=config, filename = '../django_project/static/django_project/images/html_exports/{}.html'.format(name_fig), auto_open=False)

'../django_project/static/django_project/images/html_exports/bubble_intensity_3M.html'